In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
import pandas as pd
import numpy as np
import os
import re
import gc

In [19]:
class cleaner:
    def __init__(self,df):
        self.document = None
        self.stem_func = None

# #         self.precompiled_space_re = [re.compile(r"[^A-Za-z0-9^,!.\/'+-=]>"),  # unexpected chars
# #                                      re.compile('<[\w\W]+>'),  # br tags
# #                                      re.compile(r"\s+"),
# #                                       re.compile("\d+")
# #                                     pat = re.compile(r"\d+" +str(mg)+"+.*$"+"|\d+ " +str(mg)+"+.*$")]
#         self.precompiled_space_re = [# merge white space characters, including line breaks
        self.curated_df = df
        self.malware_names = []
        self.cleaner = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

    def preprocess_document(self,amt,unit,drug):
        amt1 = str(amt)
        unit1 = str(unit).split('/')[0]
        try:
            pat = re.compile(r"(\d+"+str(unit1)+"|\d+ " +str(unit1)+ ")" + "+.*$")
            drug = re.sub(pat, " ", str(drug))
            pat1 = re.compile(r"/+[0-9]+/+.*$")
            pat2 = re.compile(r"\s+")
            drug = re.sub(pat, " ", str(drug))
            drug = re.sub(pat1, " ", str(drug))
            drug = re.sub(pat2, " ", str(drug))
            for key in self.cleaner:
                    drug =drug.replace(key, '')
        except:
            drug = drug+"_error"


        # This is specific for 20 news groups
#         if self.precompiled_space_re:
#             for cur_re in self.precompiled_space_re:
#                 document = re.sub(cur_re, " ", document)
        return drug
    def clean_column(self,name):
        self.curated_df[name + "_clean"] =  self.curated_df.apply(lambda x: self.preprocess_document(x['dose_amt'],x['dose_unit'],x['drugname']) if type(x['drugname'])!=float else x['drugname'],axis=1)
        return self.curated_df

In [20]:
class preprocess_faers():
    def __init__(self,source = None ,directory = None):
        self.source_directory = source
        self.target_directory = directory
        self.drug_df_init = pd.DataFrame(columns = ['primaryid','caseid','drugname','drugname_clean','role_cod'])
        self.demo_df_init = pd.DataFrame(columns = ['primaryid','caseid','sex','age','age_cod','age_grp','wt','wt_cod','occr_country',
                            'init_fda_dt','fda_dt','event_dt','filename'])
        self.indi_df_init = pd.DataFrame(columns = ['primaryid','caseid','indi_pt','#drugs_to_indi'])
        self.outc_df_init = pd.DataFrame(columns = ['primaryid','caseid','outc_cod'])
    def save_file(self, file_name):
        if self.target_directory==None:           
            save_file = file_name
        save_file = self.target_directory+"//"+file_name
        return save_file
    def processDrug(self,save_file=None):
        for path, subdirs, files in os.walk(self.source_directory):
            for filename in files:
                if "DRUG" in filename.upper() and "TXT" in filename.upper():
                    print(filename)
                    try:
                        drug_df = pd.read_csv(self.source_directory + "/" + filename, low_memory=False, sep="$", error_bad_lines=False)
                    except:
                        drug_df = pd.read_csv(self.source_directory + "/" + filename, low_memory=False, sep="$", 
                                              error_bad_lines=False,encoding = 'Latin-1')

                    drug_df = drug_df[drug_df['drugname'].notna()]
                    ch = cleaner(drug_df)
                    drug_df = ch.clean_column('drugname')
                    self.drug_df_init = self.drug_df_init.append(drug_df[['primaryid','caseid','drugname','drugname_clean',
                                                                          'role_cod']])
#         if self.target_directory==None:           
#             save_file = 
#         save_file = self.target_directory+"/all_drugs.csv"
        self.drug_df_init.to_csv(self.save_file("all_drugs.csv"),index = False)
    def processDemo(self,save_file=None):
        for path, subdirs, files in os.walk(self.source_directory):
            for filename in files:
                if "DEMO" in filename.upper() and "TXT" in filename.upper():
                    demo_df = pd.read_csv(self.source_directory + "/" + filename, low_memory=False, sep='$', error_bad_lines=False)
                    # save file
                    demo_df['filename'] = [filename[:-4]]*len(demo_df)
                    demo_df =demo_df[['primaryid','caseid','sex','age','age_cod','age_grp','wt','wt_cod','occr_country',
                                    'init_fda_dt','fda_dt','event_dt','filename']]
                    self.demo_df_init= self.demo_df_init.append(demo_df)
#         if self.target_directory==None:           
#             save_file = "all_demos.csv"
#         save_file = self.target_directory+"/all_demos.csv"
        self.demo_df_init.to_csv(self.save_file("all_demos.csv"),index = False)
    def processINDI(self,save_file=None):
        for path, subdirs, files in os.walk(self.source_directory):
            for filename in files:
                if "INDI" in filename.upper() and "TXT" in filename.upper():
                        indi_df = pd.read_csv(self.source_directory + "/" + filename, low_memory=False, sep='$', error_bad_lines=False)
                        indi_df = indi_df[indi_df['indi_pt'].notnull()]
                        indi_df = indi_df.groupby(['primaryid','caseid','indi_pt']).size().reset_index()
                        indi_df = indi_df.rename(columns = {0:'#drugs_to_indi'})
                        indi_df = indi_df.groupby(['primaryid','caseid']).agg(lambda x: list(x)).reset_index()
                        self.indi_df_init = self.indi_df_init.append(indi_df)
#         if self.target_directory==None:           
#             save_file = "all_indi.csv"
#         save_file = self.target_directory+"/all_indi.csv"
        self.indi_df_init.to_csv(self.save_file("all_indi.csv"),index = False)
    def processOutc(self,save_file=None):
        for path, subdirs, files in os.walk(self.source_directory):
            for filename in files:
                if "OUTC" in filename.upper() and "TXT" in filename.upper():
                    outc_df = pd.read_csv(self.source_directory + "/" + filename, low_memory=False, sep="$", error_bad_lines=False)
                    outc_df['outc_cod'] = pd.Categorical(outc_df['outc_cod'], 
                                            categories=['DE','LT','HO','DS','CA','RI','OT'], 
                                            ordered=True)
                    outc_df = outc_df.sort_values('outc_cod').groupby(['primaryid','caseid'], as_index=False).first()
                    outc_df['outc_cod'] = outc_df['outc_cod'].astype(str)
                    self.outc_df_init = self.outc_df_init.append(outc_df)
#         if self.target_directory==None:           
#             save_file = "all_outc.csv"
#         save_file = self.target_directory+"/all_outc.csv"
        self.outc_df_init.to_csv(self.save_file("all_outc.csv"),index = False)
    def preprocess(self,drug = False, demo = False, indi = False, outc = False):
        if drug==True:
            self.processDrug()
            del self.drug_df_init
            gc.collect()
        if demo== True:
            self.processDemo()
            del self.demo_df_init
            gc.collect()
        if indi== True:
            self.processINDI()
            del self.indi_df_init
            gc.collect()
        if outc ==True:
            self.processOutc()
            del self.outc_df_init
            gc.collect()
    def create_drug_features(self):
            drug_file = "all_drugs.csv"
            if self.target_directory!=None:           
                    drug_file = self.target_directory+"/all_drugs.csv"
                    alldrug_df = pd.read_csv(drug_file)
            alldrug_df = pd.read_csv(drug_file)
            alldrug_df = alldrug_df.drop(columns = ['drugname']).rename(columns = {'drugname_clean':"drugname"})
            print(alldrug_df.columns)
            ##create_most_important_drug_feature_for_baseline
            alldrug_df['role_cod'] = pd.Categorical(alldrug_df['role_cod'], 
                                            categories=['PS','SS','C','I'], 
                                            ordered=True)
            alldrug_df_baseline = alldrug_df.sort_values('role_cod').groupby(['primaryid','caseid'], as_index=False).first()
            alldrug_df_baseline = alldrug_df_baseline.reset_index()
            alldrug_df_baseline.to_csv(self.save_file("all_drugs_baseline.csv"),index = False)  
            del alldrug_df_baseline
            gc.collect()
            #create_druglist_faeture_for_tf_idf
            alldrug_df_list = alldrug_df[['primaryid','caseid','drugname','role_cod']].copy()
            del alldrug_df
            gc.collect()
            alldrug_df_listed = alldrug_df_list.groupby(['primaryid','caseid']).agg(lambda x: list(x)).reset_index()
            alldrug_df_listed = alldrug_df_listed.rename({'drugname':'drugname_listed','drugname':'role_cod_listed'})
            alldrug_df_listed.to_csv(self.save_file("all_drugs_listed.csv"),index = False)
            del alldrug_df_listed
            gc.collect()
    def create_dataset(self):
        for path, subdirs, files in os.walk(self.target_directory):
            for filename in files:
#                 print(filename)
                fileload = path + '/' + filename
                if "ALL_OUTC" in filename.upper() and "CSV" in filename.upper():
                        outc_df = pd.read_csv(fileload)
                if "ALL_DEMOS" in filename.upper() and "CSV" in filename.upper():
                        demo_df = pd.read_csv(fileload)
                if "ALL_INDI" in filename.upper() and "CSV" in filename.upper():
                        indi_df = pd.read_csv(fileload)
                if "ALL_DRUGS_BASELINE" in filename.upper() and "CSV" in filename.upper():
                        drug_base = pd.read_csv(fileload)
                if "ALL_DRUGS_LISTED" in filename.upper() and "CSV" in filename.upper():
                        drug_listed = pd.read_csv(fileload)
        drug_listed = drug_listed.rename(columns = {'drugname':'drugname_listed','role_cod':'role_cod_listed'})
        demo_outc = pd.merge(outc_df,demo_df,how = 'left', on=['primaryid','caseid'])
        demo_outc = pd.merge(demo_outc,indi_df,how = 'left', on=['primaryid','caseid'])
        demo_outc = pd.merge(demo_outc,drug_base,how = 'left', on=['primaryid','caseid'])
        dataset = pd.merge(demo_outc,drug_listed,how = 'left', on=['primaryid','caseid'])
        if self.target_directory==None:           
            save_file = "dataset.csv"
        save_file = self.target_directory+"/dataset.csv"
        dataset.to_csv(self.save_file("dataset.csv"),index = False)
        return demo_outc

In [21]:
source = '/content/drive/MyDrive/Colab Notebooks/Project/ASCII'
target = '/content/drive/MyDrive/Colab Notebooks/Project/preprocessed_files'
preprocess_class = preprocess_faers(source,target)

In [22]:
# preprocess_class.preprocess(demo = True)
# preprocess_class.preprocess(indi = True)
# preprocess_class.preprocess(outc = True)
# preprocess_class.preprocess(drug = True)
# preprocess_class.create_drug_features()
preprocess_class.create_dataset()

,primaryid,caseid,outc_cod,sex,age,age_cod,age_grp,wt,wt_cod,occr_country,init_fda_dt,fda_dt,event_dt,filename,indi_pt,#drugs_to_indi,index,drugname,role_cod
0,36362202,3636220,OT,NaN,NaN,NaN,N,NaN,NaN,NL,20010405,20210902,NaN,DEMO21Q3,['Foetal exposure during pregnancy'],[1],35.0,OXCARBAZEPINE,PS
1,41505154,4150515,OT,F,56.0,YR,NaN,56.0,KG,FR,20040607,20210712,200203.0,DEMO21Q3,"['Dyslipidaemia', 'HIV infection']","[1, 4]",364.0,EFAVIRENZ,PS
2,58189452,5818945,HO,M,50.0,YR,NaN,NaN,NaN,IT,20050615,20210715,199811.0,DEMO21Q3,"['Immunosuppressant drug therapy', 'Renal tran...","[4, 4]",682.0,CYCLOSPORINE,PS
3,59253972,5925397,HO,M,28.0,YR,A,NaN,NaN,US,20051118,20210706,NaN,DEMO21Q3,['Staphylococcal infection'],[2],813.0,RIFAMPICIN,PS
4,60937133,6093713,OT,F,52.0,YR,NaN,NaN,NaN,ES,20060719,20210809,NaN,DEMO21Q3,"['Hiatus hernia', 'Hypercholesterolaemia']","[1, 1]",1061.0,OMEPRAZOLE,PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6048451,1057854810,10578548,HO,F,48.0,YR,NaN,83.9,KG,US,20141112,20150211,2014.0,DEMO15Q1,"['Product used for unknown indication', 'Pulmo...","[2, 1]",10554338.0,REMODULIN,PS
6048452,1059001410,10590014,HO,F,72.0,YR,NaN,61.0,KG,US,20141118,20150319,20141103.0,DEMO15Q1,['Rheumatoid arthritis'],[4],10554435.0,TOFACITINIB CITRATE,PS
6048453,1064655210,10646552,HO,M,53.0,YR,NaN,71.0,KG,US,20141211,20150309,20141125.0,DEMO15Q1,"['Depression', 'Renal cell carcinoma']","[1, 2]",10554789.0,PEMBROLIZUMAB,PS
6048454,1066547310,10665473,DE,F,56.0,YR,NaN,NaN,NaN,GB,20141219,20150320,20141204.0,DEMO15Q1,['Schizophrenia'],[1],10554888.0,CLOZARIL,PS
